# Ingesting PDF Documents

## Setup Azure API Keys

In [1]:
from notebookutils.mssparkutils.credentials import getSecret

KEYVAULT_ENDPOINT = "https://rag-demo-east-us-kv.vault.azure.net/"
# Azure AI Search
AI_SEARCH_NAME = getSecret(KEYVAULT_ENDPOINT, "AI-SEARCH-NAME")
AI_SEARCH_API_KEY = getSecret(KEYVAULT_ENDPOINT, "AI-SEARCH-API-KEY")
AI_SEARCH_INDEX_NAME = "rag-demo-index"
# Azure AI Services
AI_SERVICES_NAME = getSecret(KEYVAULT_ENDPOINT, "AI-SERVICES-NAME")
AI_SERVICES_API_KEY = getSecret(KEYVAULT_ENDPOINT, "AI-SERVICES-API-KEY")
AI_SERVICES_LOCATION = "eastus"
# Azure Open AI - (if F64 SKU is not used)
OPEN_AI_NAME = getSecret(KEYVAULT_ENDPOINT, "OPEN-AI-NAME")
OPEN_AI_API_KEY = getSecret(KEYVAULT_ENDPOINT, "OPEN-AI-API-KEY")
#OPEN_AI_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-ada-002" #1536
#OPEN_AI_GPT_DEPLOYMENT_NAME = "gpt-35-turbo-16k" # deploymentName could be one of {gpt-35-turbo, gpt-35-turbo-16k}

StatementMeta(, 4345724c-8ad5-4246-a1d4-66302b187e3a, 3, Finished, Available)

## Github: Do not embed secrets within the code
![Github secrets commit error](https://github.com/luisdza/rag-fabric-workbook/raw/main/images/github-commit.png)

## Load and Analyse the Document

In [2]:
import requests
import os

url = "https://github.com/luisdza/rag-fabric-workbook/raw/main/docs/Northwind_Standard_Benefits_Details.pdf"
#url = "https://github.com/luisdza/rag-fabric-workbook/raw/main/docs/Northwind_Health_Plus_Benefits_Details.pdf"

response = requests.get(url)

# Specify your path here
path = "/lakehouse/default/Files/"

# Ensure the directory exists
os.makedirs(path, exist_ok=True)

# Write the content to a file in the specified path
filename = url.rsplit("/")[-1]
with open(os.path.join(path, filename), "wb") as f:
    f.write(response.content)

StatementMeta(, 4345724c-8ad5-4246-a1d4-66302b187e3a, 4, Finished, Available)

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

document_path = f"Files/{filename}"

df = spark.read.format("binaryFile").load(document_path).select("_metadata.file_name", "content").limit(10).cache()

display(df)


StatementMeta(, 4345724c-8ad5-4246-a1d4-66302b187e3a, 5, Finished, Available)

Py4JJavaError: An error occurred while calling o4359.load.
: Operation failed: "Bad Request", 400, HEAD, http://onelake.dfs.fabric.microsoft.com/a839d788-eec3-446b-bf34-7aa9e3479a54/user/trusted-service-user/Files/Northwind_Standard_Benefits_Details.pdf?upn=false&action=getStatus&timeout=90
	at org.apache.hadoop.fs.azurebfs.services.AbfsRestOperation.completeExecute(AbfsRestOperation.java:231)
	at org.apache.hadoop.fs.azurebfs.services.AbfsRestOperation.lambda$execute$0(AbfsRestOperation.java:191)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.trackDurationOfInvocation(IOStatisticsBinding.java:464)
	at org.apache.hadoop.fs.azurebfs.services.AbfsRestOperation.execute(AbfsRestOperation.java:189)
	at org.apache.hadoop.fs.azurebfs.services.AbfsClient.getPathStatus(AbfsClient.java:690)
	at org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.getFileStatus(AzureBlobFileSystemStore.java:1046)
	at org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.getFileStatus(AzureBlobFileSyst

In [ ]:
from synapse.ml.services import AnalyzeDocument
from pyspark.sql.functions import col

analyze_document = (
    AnalyzeDocument()
    .setPrebuiltModelId("prebuilt-layout")
    .setCustomServiceName(AI_SERVICES_NAME)
    .setSubscriptionKey(AI_SERVICES_API_KEY)
    .setLocation(AI_SERVICES_LOCATION)
    .setImageBytesCol("content")
    .setOutputCol("result")
)

analyzed_df = (
    analyze_document.transform(df)
    .withColumn("output_content", col("result.analyzeResult.content"))
    .withColumn("paragraphs", col("result.analyzeResult.paragraphs"))
).cache()

StatementMeta(, , , Cancelled, )

In [ ]:
analyzed_df = analyzed_df.drop("content")
display(analyzed_df)

StatementMeta(, , , Cancelled, )

## Chunking Text

In [ ]:
from synapse.ml.featurize.text import PageSplitter

ps = (
    PageSplitter()
    .setInputCol("output_content")
    .setMaximumPageLength(4000)
    .setMinimumPageLength(3000)
    .setOutputCol("chunks")
)

splitted_df = ps.transform(analyzed_df)
display(splitted_df)

StatementMeta(, , , Cancelled, )

In [ ]:
from pyspark.sql.functions import posexplode, col, concat

# Each "chunks" column contains the chunks for a single document in an array
# The posexplode function will separate each chunk into its own row
exploded_df = splitted_df.select("file_name", posexplode(col("chunks")).alias("chunk_index", "chunk"))

# Add a unique identifier for each chunk
exploded_df = exploded_df.withColumn("unique_id", concat(exploded_df.file_name, exploded_df.chunk_index))

# Write the exploded_df DataFrame to a Lakehouse table in Microsoft Fabric
exploded_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("rag_demo.document_chunks")

display(exploded_df)

StatementMeta(, , , Cancelled, )